In [1]:
import sys
sys.path.insert(0,'..')

import yaml
import os
from yaml import Loader as Loader
from pathlib import Path
import pandas as pd
import numpy as np
import json
from sklearn.metrics import roc_auc_score

from analyze_results import (
    extract_result,
    aggregate_runs,
    from_model_outputs_calc_rcc_auc,
    format_results2,
    improvement_over_baseline,
    from_model_outputs_calc_pr_auc,
    from_model_outputs_calc_rpp,
    from_model_outputs_calc_roc_auc,
    from_model_outputs_calc_arc_auc,
    from_model_outputs_calc_f1_score,
    from_model_outputs_calc_recall,
    from_model_outputs_calc_precision
)

from utils.utils_wandb import init_wandb, wandb
from ue4nlp.ue_scores import *

In [2]:
def choose_metric(metric_type):
    if metric_type  == "rejection-curve-auc":
        return metric_type
    elif metric_type  == "roc-auc":
        return metric_type
    elif metric_type == "rcc-auc":
        return from_model_outputs_calc_rcc_auc
    elif metric_type == "rpp":
        return from_model_outputs_calc_rpp
    elif metric_type == "pr-auc":
        return from_model_outputs_calc_pr_auc
    elif metric_type == "f1-score":
        return from_model_outputs_calc_f1_score
    elif metric_type == "recall":
        return from_model_outputs_calc_recall
    elif metric_type == "precision":
        return from_model_outputs_calc_precision
    else:
        raise ValueError("Wrong metric type!")
        
def get_one_table(runs_dir, metric_types=["rejection-curve-auc", "roc-auc", "pr-auc"], baseline=None, methods=None):
    default_methods = {
        "bald": bald,
        "sampled_max_prob": sampled_max_prob,
        "variance": probability_variance,
    }
    
    if methods is None:
        methods = default_methods

    table = []
    for metric_type in metric_types:
        metric = choose_metric(metric_type=metric_type)

        agg_res = aggregate_runs(
            runs_dir, methods=methods, metric=metric, oos=True
        )
        if agg_res.empty:
            print("Broken\n")
            continue

        if metric_type == "rcc-auc":
            final_score = format_results2(agg_res, percents=False)
        elif metric_type == "rpp":
            final_score = format_results2(agg_res, percents=True)
        elif metric_type == "accuracy":
            final_score = format_results2(agg_res, percents=True)
        else:
            final_score = improvement_over_baseline(agg_res, baseline_col="max_prob", baseline=baseline, metric=metric_type, percents=True, subtract=False)
        table.append(final_score)
    res_table = pd.concat(table, axis=1)
    res_table.columns = metric_types
    # fix for rcc-auc and rpp
    if 'baseline (max_prob)' not in res_table.index:
        res_table.loc['baseline (max_prob)'] = 0
    try:
        res_table = res_table.drop(['max_prob', 'count'])
    except:
        res_table = res_table.drop(['count'])
    return res_table


def collect_tables(run_dirs, names, metric_types=["rejection-curve-auc", "roc-auc", "rcc-auc", "pr-auc", "rpp"], baseline=None, methods=None):
    all_tables = []
    for run_dir, name in zip(run_dirs, names):
        buf_table = get_one_table(run_dir, metric_types, baseline, methods)
        #print(buf_table)
        # add name to index
        indices = [(name, ind) for ind in list(buf_table.index)]
        baseline_name = 'baseline|'+'|'.join(name.split('|')[1:])
        buf_table.loc[baseline_name] = buf_table.loc['baseline (max_prob)']
        # add reindex
        indices = indices + [(baseline_name, 'max_prob')]
        
        index = pd.MultiIndex.from_tuples(indices, names=['Method', 'UE Score'])
        buf_table.index = index
        buf_table.drop((name, 'baseline (max_prob)'), inplace=True)
        # add buf_table to final_table
        all_tables.append(buf_table)
    return pd.concat(all_tables)


def collect_datasets(runs_dirs, names, dataset_names, metric_types=["rejection-curve-auc", "roc-auc", "rcc-auc", "pr-auc", "rpp"], baselines={}, methods=None):
    all_tables = []
    for run_dir, dataset_name in zip(runs_dirs, dataset_names):
        try:
            dataset_table = collect_tables(run_dir, names, metric_types, baselines.get(dataset_name, None), methods=methods)
            columns = pd.MultiIndex.from_tuples([(dataset_name, ind) for ind in list(dataset_table.columns)])
            dataset_table.columns = columns
            all_tables.append(dataset_table)
        except:
            print(f'empty dir {run_dir}')
    return pd.concat(all_tables, axis=1)

# OOD datasets

In [21]:
import os 
def choose_agg_func(method):
    agg_methods = {
        "bald": bald,
        "sampled_max_prob": sampled_max_prob,
        "variance": probability_variance,
    }
    if method=='nuq' or method=='nuq_best' or method=='nuq_best1':
        nuq_aleatoric = lambda x: np.squeeze(x[0], axis=-1)
        nuq_epistemic = lambda x: np.squeeze(x[1], axis=-1)
        nuq_total = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {
            "nuq_aleatoric": nuq_aleatoric,
            "nuq_epistemic": nuq_epistemic,
            "nuq_total": nuq_total,
        }
    elif method=='decomposing_md':
        disc_md = lambda x: np.squeeze(x[0], axis=-1)
        nondisc_md = lambda x: np.squeeze(x[1], axis=-1)
        sum_md = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {"disc_md": disc_md, 
                       'nondisc_md': nondisc_md,
                       'disc+nondisc_md': sum_md}
    elif method=='mahalanobis':
        maha_dist = lambda x: np.squeeze(x[0], axis=-1)
        rel_maha_dist = lambda x: np.squeeze(x[1], axis=-1)
        marg_maha_dist = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {"mahalanobis_distance": maha_dist,
                       "relative_mahalanobis_distance": rel_maha_dist,
                       "marginal_mahalanobis_distance": marg_maha_dist}
    elif method=='ddu' or method=='ddu_maha':
        ddu = lambda x: -np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"ddu": ddu}
    elif method=='sngp':
        sngp = lambda x: np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"sngp": sngp}
    elif method=='mc_mahalanobis':
        sm_maha_dist = lambda x: np.squeeze(x[:, 1:], axis=-1).max(1)
        agg_methods = {"sampled_mahalanobis_distance": sm_maha_dist}
    return agg_methods
metric_types=["roc-auc", 'pr-auc']

methods = ['nuq', 'mahalanobis', 'decomposing_md', 'ddu']#, 'sngp']
regs = ['raw', 'reg', 'metric']
spectralnorm = ['sn', 'no_sn']
dataset_names = ['CLINC', 'ROSTD']
dataset_fnames = ['clinc', 'rostd']
names = []
tables = []
baselines = []
for method in methods:
    for reg in regs:
        for sn in spectralnorm:
            run_dirs = []
            name_sn = ''
            names = [f'{method}|{reg}|{sn}']
            for name in dataset_fnames:
                model_series_dir = f'../../workdir/run_glue_for_model_series/electra_{reg}_{sn}/{name}/{method}'
                print(model_series_dir)
                run_dirs.append([model_series_dir])
            agg_func = choose_agg_func(method)
            try:
                res_df = collect_datasets(run_dirs, names, dataset_names, metric_types=metric_types, baselines={}, methods=agg_func)
                baselines.append(res_df.iloc[-1:])
                tables.append(res_df.iloc[:-1])
            except:
                print('pass')
                pass

../../workdir/run_glue_for_model_series/electra_raw_sn/clinc/nuq
../../workdir/run_glue_for_model_series/electra_raw_sn/rostd/nuq
empty dir ['../../workdir/run_glue_for_model_series/electra_raw_sn/clinc/nuq']
empty dir ['../../workdir/run_glue_for_model_series/electra_raw_sn/rostd/nuq']
pass
../../workdir/run_glue_for_model_series/electra_raw_no_sn/clinc/nuq
../../workdir/run_glue_for_model_series/electra_raw_no_sn/rostd/nuq
empty dir ['../../workdir/run_glue_for_model_series/electra_raw_no_sn/clinc/nuq']
empty dir ['../../workdir/run_glue_for_model_series/electra_raw_no_sn/rostd/nuq']
pass
../../workdir/run_glue_for_model_series/electra_reg_sn/clinc/nuq
../../workdir/run_glue_for_model_series/electra_reg_sn/rostd/nuq
empty dir ['../../workdir/run_glue_for_model_series/electra_reg_sn/clinc/nuq']
empty dir ['../../workdir/run_glue_for_model_series/electra_reg_sn/rostd/nuq']
pass
../../workdir/run_glue_for_model_series/electra_reg_no_sn/clinc/nuq
../../workdir/run_glue_for_model_series/e

In [4]:
table_all = pd.concat([pd.concat(tables), pd.concat(baselines[:2])]).reset_index()

In [5]:
def preproc_regs(x):
    reg = x.split('|')[1]
    if reg == 'reg':
        return 'CER'
    elif reg == 'raw':
        return '-'
    else:
        return reg
    
def preproc_method(x):
    method = x.split('|')[0]
    sn = x.split('|')[-1]
    if method == 'mahalanobis' and not 'no_sn' in sn:
        return 'MD SN (ours)'
    elif method == 'mahalanobis':
        return 'MD'
    elif method == 'mc_mahalanobis' and not 'no_sn' in sn:
        return 'SMD SN (ours)'
    elif method == 'mc_mahalanobis':
        return 'SMD'
    elif method == 'nuq' and not 'no_sn' in sn:
        return 'NUQ SN'
    elif method == 'nuq':
        return 'NUQ'
    elif method == 'sngp':
        return 'SNGP'
    
    elif method == 'decomposing_md' and not 'no_sn' in sn:
        return 'Decomposing SN'
    elif method == 'decomposing_md':
        return 'Decomposing'
    
    elif method == 'nuq_best1' and not 'no_sn' in sn:
        return 'Best1 NUQ SN'
    elif method == 'nuq_best1':
        return 'Best1 NUQ'
    
    elif method == 'ddu' and not 'no_sn' in sn:
        return 'DDU SN'
    elif method == 'ddu':
        return 'DDU'
    
    elif method == 'ddu' and not 'no_sn' in sn:
        return 'DDU SN'
    elif method == 'ddu':
        return 'DDU'
    
    elif method == 'ddu_maha' and not 'no_sn' in sn:
        return 'DDU Maha SN'
    elif method == 'ddu_maha':
        return 'DDU Maha'
    
    elif 'ddpp_dpp' in method:
        return 'DDPP (+DPP) (ours)'
    elif 'ddpp_ood' in method:
        return 'DDPP (+OOD) (ours)'
    elif 'mc_all' in method:
        return 'MC dropout'
    elif 'Deep' in method:
        return 'Deep Ensemble'
    elif 'baseline|raw_no_sn' in x:
        return 'SR (baseline)'
    elif 'baseline' in x and not 'no_sn' in x:
        return 'SR SN'
    return 'SR'

def preproc_ue(x):
    if x == 'bald':
        return 'BALD'
    elif 'sampled_mahalanobis_distance' in x:
        return 'SMD'
    elif 'relative_mahalanobis_distance' in x:
        return 'rel_MD'
    elif 'marginal_mahalanobis_distance' in x:
        return 'marg_MD'
    elif 'mahalanobis_distance' in x:
        return 'MD'
    elif 'sampled_max_prob' in x:
        return 'SMP'
    elif 'variance' in x:
        return 'PV'
    elif 'aleatoric' in x:
        return 'aleatoric'
    elif 'epistemic' in x:
        return 'epistemic'
    elif 'total' in x:
        return 'total'
    elif x == 'disc_md':
        return 'Disc MD'
    elif x == 'nondisc_md':
        return 'Nondisc MD'
    elif x == 'disc+nondisc_md':
        return 'Disc+Nondisc MD'
    elif x == 'sngp':
        return 'std'
    return 'MP'

table_all['Reg. Type'] = table_all.Method.apply(lambda x: preproc_regs(x))
table_all['Method'] = table_all.Method.apply(lambda x: preproc_method(x))
table_all['UE Score'] = table_all['UE Score'].apply(lambda x: preproc_ue(x))
table_all = table_all[list(table_all.columns[:1]) + list(table_all.columns[-1:]) + list(table_all.columns[1:-1])].reset_index(drop=True)

In [8]:
table_all.iloc[:-2]

Method Reg. Type         UE Score     CLINC                 ROSTD  \
                                                 pr-auc    roc-auc     pr-auc   
0           NUQ SN         -        aleatoric  81.0±2.7  66.3±26.5   95.2±0.9   
1           NUQ SN         -        epistemic  86.5±1.7   95.1±0.9   78.3±5.8   
2           NUQ SN         -            total  82.7±2.0  77.9±17.3   84.8±4.3   
3              NUQ         -        aleatoric  79.6±3.0  53.3±26.2   95.9±0.4   
4              NUQ         -        epistemic  87.1±0.4   95.9±0.5   82.4±3.2   
5              NUQ         -            total  81.5±2.5  70.7±16.1   88.6±3.2   
6           NUQ SN       CER        aleatoric  79.6±2.1  54.8±18.9   95.6±1.3   
7           NUQ SN       CER        epistemic  87.2±0.7   95.1±0.5   81.6±6.9   
8           NUQ SN       CER            total  81.8±1.6  71.9±12.2   87.5±5.6   
9              NUQ       CER        aleatoric  80.3±1.7  62.1±17.5   92.9±3.3   
10             NUQ       CER        epistemic  87.4±0.3   96.2±0.4   72.0±7.8   
11             NUQ       CER            total  82.1±1.5   77.4±9.5   79.8±6.4   
12          NUQ SN    metric        aleatoric       NaN        NaN   95.1±1.7   
13          NUQ SN    metric        epistemic       NaN        NaN   81.6±3.4   
14          NUQ SN    metric            total       NaN        NaN   87.3±3.5   
15             NUQ    metric        aleatoric       NaN        NaN   90.8±1.9   
16             NUQ    metric        epistemic       NaN        NaN   66.7±5.4   
17             NUQ    metric            total       NaN        NaN   77.0±4.3   
18    MD SN (ours)         -               MD  87.6±0.5   97.1±0.1   98.4±0.3   
19    MD SN (ours)         -           rel_MD  87.4±0.5   97.0±0.1   93.2±9.5   
20    MD SN (ours)         -          marg_MD  87.6±0.4   97.3±0.2   98.4±0.3   
21              MD         -               MD  87.5±0.3   97.3±0.2   98.2±0.2   
22              MD         -           rel_MD  87.3±0.3   97.1±0.1  89.0±16.6   
23              MD         -          marg_MD  87.6±0.3   97.4±0.2   98.2±0.2   
24    MD SN (ours)       CER               MD  87.9±0.4   97.2±0.2   98.5±0.2   
25    MD SN (ours)       CER           rel_MD  87.7±0.5   97.0±0.1   96.5±1.4   
26    MD SN (ours)       CER          marg_MD  88.0±0.4   97.3±0.2   98.5±0.2   
27              MD       CER               MD  87.8±0.3   97.3±0.1   98.0±0.3   
28              MD       CER           rel_MD  87.6±0.3   97.2±0.2  85.2±13.3   
29              MD       CER          marg_MD  87.8±0.3   97.4±0.1   98.0±0.3   
30    MD SN (ours)    metric               MD       NaN        NaN   98.7±0.4   
31    MD SN (ours)    metric           rel_MD       NaN        NaN   93.7±4.8   
32    MD SN (ours)    metric          marg_MD       NaN        NaN   98.5±0.4   
33              MD    metric               MD       NaN        NaN   96.7±1.1   
34              MD    metric           rel_MD       NaN        NaN  80.8±11.4   
35              MD    metric          marg_MD       NaN        NaN   96.6±1.2   
36  Decomposing SN         -          Disc MD  87.6±0.5   96.9±0.1   96.0±1.0   
37  Decomposing SN         -       Nondisc MD  87.6±0.4   96.9±0.1   96.4±0.9   
38  Decomposing SN         -  Disc+Nondisc MD  87.6±0.5   96.9±0.1   96.6±0.8   
39     Decomposing         -          Disc MD  87.6±0.4   97.0±0.2   96.0±0.9   
40     Decomposing         -       Nondisc MD  87.6±0.3   97.1±0.2   96.3±1.0   
41     Decomposing         -  Disc+Nondisc MD  87.6±0.4   97.1±0.2   96.5±0.8   
42  Decomposing SN       CER          Disc MD  87.9±0.6   96.9±0.1   96.3±0.4   
43  Decomposing SN       CER       Nondisc MD  88.0±0.5   96.9±0.1   96.2±0.5   
44  Decomposing SN       CER  Disc+Nondisc MD  88.0±0.5   96.9±0.1   96.6±0.3   
45     Decomposing       CER          Disc MD  87.7±0.3   97.1±0.1   95.0±0.6   
46     Decomposing       CER       Nondisc MD  87.7±0.3   97.1±0.1   94.7±1.3   
47     Decomposing       CER  Disc+Nondisc 

In [9]:
table_all.iloc[-2:]

Method Reg. Type UE Score     CLINC               ROSTD          
                                pr-auc   roc-auc    pr-auc   roc-auc
60  SR SN         -       MP  86.0±0.9  94.3±0.4  68.4±4.2  77.0±3.2
61     SR         -       MP  85.9±0.5  95.7±0.2  70.9±5.0  78.7±3.5

# OOD new Benchmark

## SST-2

In [17]:
import os 

def choose_agg_func(method):
    agg_methods = {
        "bald": bald,
        "sampled_max_prob": sampled_max_prob,
        "variance": probability_variance,
    }
    if method=='nuq' or method=='nuq_best' or method=='nuq_best1':
        nuq_aleatoric = lambda x: np.squeeze(x[0], axis=-1)
        nuq_epistemic = lambda x: np.squeeze(x[1], axis=-1)
        nuq_total = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {
            "nuq_aleatoric": nuq_aleatoric,
            "nuq_epistemic": nuq_epistemic,
            "nuq_total": nuq_total,
        }
    elif method=='l_nuq':
        nuq_aleatoric = lambda x: np.squeeze(x[0], axis=-1)
        nuq_epistemic = lambda x: np.squeeze(x[1], axis=-1)
        nuq_total = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {
            "l_nuq_aleatoric": nuq_aleatoric,
            "l_nuq_epistemic": nuq_epistemic,
            "l_nuq_total": nuq_total,
        }
    elif method=='decomposing_md':
        disc_md = lambda x: np.squeeze(x[0], axis=-1)
        nondisc_md = lambda x: np.squeeze(x[1], axis=-1)
        sum_md = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {"disc_md": disc_md, 
                       'nondisc_md': nondisc_md,
                       'disc+nondisc_md': sum_md}
    elif method=='mahalanobis':
        maha_dist = lambda x: np.squeeze(x[0], axis=-1)
        rel_maha_dist = lambda x: np.squeeze(x[1], axis=-1)
        marg_maha_dist = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {"mahalanobis_distance": maha_dist,
                       "relative_mahalanobis_distance": rel_maha_dist,
                       "marginal_mahalanobis_distance": marg_maha_dist}
    elif method=='l_mahalanobis':
        maha_dist = lambda x: np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"l_mahalanobis_distance": maha_dist}
    elif method=='ddu' or method=='ddu_maha':
        ddu = lambda x: -np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"ddu": ddu}
    elif method=='sngp':
        sngp = lambda x: np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"sngp": sngp}
    elif method=='mc_mahalanobis':
        sm_maha_dist = lambda x: np.squeeze(x[:, 1:], axis=-1).max(1)
        agg_methods = {"sampled_mahalanobis_distance": sm_maha_dist}
    return agg_methods
metric_types=["roc-auc", 'pr-auc', 'f1-score', 'precision', 'recall']

methods = ['l_nuq', 'nuq', 'mahalanobis', 'l_mahalanobis', 'decomposing_md', 'ddu']
regs = ['raw']
spectralnorm = ['sn', 'no_sn']
dataset_names = ['IMDB', 'TREC', 'WMT16', 'Amazon', 'MNLI', '20newsgroups', 'RTE']
dataset_fnames = ['imdb', 'trec', 'wmt16', 'amazon', 'mnli', 'newsgroup', 'rte']
names = []
tables = []
baselines = []
for method in methods:
    for reg in regs:
        for sn in spectralnorm:
            run_dirs = []
            name_sn = ''
            names = [f'{method}|{reg}|{sn}']
            for name in dataset_fnames:
                model_series_dir = f'../../workdir/run_tasks_for_model_series/electra_{reg}_{sn}/sst2/{name}/{method}'
                print(model_series_dir)
                run_dirs.append([model_series_dir])
            agg_func = choose_agg_func(method)
            try:
                res_df = collect_datasets(run_dirs, names, dataset_names, metric_types=metric_types, baselines={}, methods=agg_func)
                baselines.append(res_df.iloc[-1:])
                tables.append(res_df.iloc[:-1])
            except:
                print('pass')
                pass

../../workdir/run_tasks_for_model_series/electra_raw_sn/sst2/imdb/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn/sst2/trec/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn/sst2/wmt16/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn/sst2/amazon/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn/sst2/mnli/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn/sst2/newsgroup/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn/sst2/rte/l_nuq
empty dir ['../../workdir/run_tasks_for_model_series/electra_raw_sn/sst2/imdb/l_nuq']
empty dir ['../../workdir/run_tasks_for_model_series/electra_raw_sn/sst2/trec/l_nuq']
empty dir ['../../workdir/run_tasks_for_model_series/electra_raw_sn/sst2/wmt16/l_nuq']
empty dir ['../../workdir/run_tasks_for_model_series/electra_raw_sn/sst2/amazon/l_nuq']
empty dir ['../../workdir/run_tasks_for_model_series/electra_raw_sn/sst2/mnli/l_nuq']
empty dir ['../../workdir/run_tasks_for_model_series/ele

In [18]:
table_all = pd.concat([pd.concat(tables), pd.concat(baselines[-2:])]).reset_index()

ValueError: No objects to concatenate

In [19]:
def preproc_regs(x):
    reg = x.split('|')[1]
    if reg == 'reg':
        return 'CER'
    elif reg == 'raw':
        return '-'
    else:
        return reg
    
def preproc_method(x):
    method = x.split('|')[0]
    sn = x.split('|')[-1]
    if method == 'mahalanobis' and not 'no_sn' in sn:
        return 'MD SN (ours)'
    elif method == 'mahalanobis':
        return 'MD'
    elif method == 'l_mahalanobis' and not 'no_sn' in sn:
        return 'L-MD SN (ours)'
    elif method == 'l_mahalanobis':
        return 'L-MD'
    elif method == 'mc_mahalanobis' and not 'no_sn' in sn:
        return 'SMD SN (ours)'
    elif method == 'mc_mahalanobis':
        return 'SMD'
    elif method == 'nuq' and not 'no_sn' in sn:
        return 'NUQ SN'
    elif method == 'nuq':
        return 'NUQ'
    elif method == 'l_nuq' and not 'no_sn' in sn:
        return 'L-NUQ SN'
    elif method == 'l_nuq':
        return 'L-NUQ'
    elif method == 'sngp':
        return 'SNGP'
    
    elif method == 'decomposing_md' and not 'no_sn' in sn:
        return 'Decomposing SN'
    elif method == 'decomposing_md':
        return 'Decomposing'
    
    elif method == 'nuq_best1' and not 'no_sn' in sn:
        return 'Best1 NUQ SN'
    elif method == 'nuq_best1':
        return 'Best1 NUQ'
    
    elif method == 'ddu' and not 'no_sn' in sn:
        return 'DDU SN'
    elif method == 'ddu':
        return 'DDU'
    
    elif method == 'ddu' and not 'no_sn' in sn:
        return 'DDU SN'
    elif method == 'ddu':
        return 'DDU'
    
    elif method == 'ddu_maha' and not 'no_sn' in sn:
        return 'DDU Maha SN'
    elif method == 'ddu_maha':
        return 'DDU Maha'
    
    elif 'ddpp_dpp' in method:
        return 'DDPP (+DPP) (ours)'
    elif 'ddpp_ood' in method:
        return 'DDPP (+OOD) (ours)'
    elif 'mc_all' in method:
        return 'MC dropout'
    elif 'Deep' in method:
        return 'Deep Ensemble'
    elif 'baseline|raw_no_sn' in x:
        return 'SR (baseline)'
    elif 'baseline' in x and not 'no_sn' in x:
        return 'SR SN'
    return 'SR'

def preproc_ue(x):
    if x == 'bald':
        return 'BALD'
    elif 'sampled_mahalanobis_distance' in x:
        return 'SMD'
    elif 'relative_mahalanobis_distance' in x:
        return 'rel_MD'
    elif 'marginal_mahalanobis_distance' in x:
        return 'marg_MD'
    elif 'mahalanobis_distance' in x:
        return 'MD'
    elif 'sampled_max_prob' in x:
        return 'SMP'
    elif 'variance' in x:
        return 'PV'
    elif 'aleatoric' in x:
        return 'aleatoric'
    elif 'epistemic' in x:
        return 'epistemic'
    elif 'total' in x:
        return 'total'
    elif x == 'disc_md':
        return 'Disc MD'
    elif x == 'nondisc_md':
        return 'Nondisc MD'
    elif x == 'disc+nondisc_md':
        return 'Disc+Nondisc MD'
    elif x == 'sngp':
        return 'std'
    return 'MP'

table_all['Reg. Type'] = table_all.Method.apply(lambda x: preproc_regs(x))
table_all['Method'] = table_all.Method.apply(lambda x: preproc_method(x))
table_all['UE Score'] = table_all['UE Score'].apply(lambda x: preproc_ue(x))
table_all = table_all[list(table_all.columns[:1]) + list(table_all.columns[-1:]) + list(table_all.columns[1:-1])].reset_index(drop=True)

IndexError: list index out of range

In [20]:
roc_auc_mean = table_all[table_all.columns[3::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
roc_auc_std = table_all[table_all.columns[3::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

f1_mean = table_all[table_all.columns[5::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
f1_std = table_all[table_all.columns[5::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

pr_auc_mean = table_all[table_all.columns[4::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
pr_auc_std = table_all[table_all.columns[4::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

pr_mean = table_all[table_all.columns[6::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
pr_std = table_all[table_all.columns[6::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

rec_auc_mean = table_all[table_all.columns[7::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
rec_auc_std = table_all[table_all.columns[7::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

In [27]:
table_all[('Mean',  'pr-auc')] = pr_auc_mean.round(2).astype(str)+'±'+pr_auc_std.round(2).astype(str)
table_all[('Mean',  'f1-score')] = f1_mean.round(2).astype(str)+'±'+f1_std.round(2).astype(str)
table_all[('Mean',  'precision')] = pr_mean.round(2).astype(str)+'±'+pr_std.round(2).astype(str)
table_all[('Mean',  'recall')] = rec_auc_mean.round(2).astype(str)+'±'+rec_auc_std.round(2).astype(str)
table_all[('Mean',  'roc-auc')] = roc_auc_mean.round(2).astype(str)+'±'+roc_auc_std.round(2).astype(str)

In [28]:
def bold_max(table):
    attr = 'font-weight: bold'
    data = table[table.columns[3:]].apply(lambda x: x.str.split('±').str[0].astype(float))
    is_max = data == data.max()
    info_col = table[table.columns[:3]]
    for col in info_col.columns:
        info_col[col] = ''
    vals = pd.DataFrame(np.where(is_max, attr, ''),
                        index=data.index, columns=data.columns)
    return pd.concat([info_col, vals], axis=1)

def highlight_nmax(s):
    try:
        s_vals = s.str.split('±').str[0].astype(float)
        is_large = s_vals.nlargest(3).values
        return ['background-color: yellow' if v in is_large else '' for v in s_vals]
    except:
        return ['']*len(s)

In [29]:
table_all[list(table_all.columns[:3]) + list(table_all.columns[-5:])].style.apply(highlight_nmax).apply(bold_max, axis=None)

/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [30]:
table_all.style.apply(highlight_nmax).apply(bold_max, axis=None)

/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


## 20newsgroups

In [31]:
import os 
def choose_agg_func(method):
    agg_methods = {
        "bald": bald,
        "sampled_max_prob": sampled_max_prob,
        "variance": probability_variance,
    }
    if method=='nuq' or method=='nuq_best' or method=='nuq_best1':
        nuq_aleatoric = lambda x: np.squeeze(x[0], axis=-1)
        nuq_epistemic = lambda x: np.squeeze(x[1], axis=-1)
        nuq_total = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {
            "nuq_aleatoric": nuq_aleatoric,
            "nuq_epistemic": nuq_epistemic,
            "nuq_total": nuq_total,
        }
    elif method=='l_nuq':
        nuq_aleatoric = lambda x: np.squeeze(x[0], axis=-1)
        nuq_epistemic = lambda x: np.squeeze(x[1], axis=-1)
        nuq_total = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {
            "l_nuq_aleatoric": nuq_aleatoric,
            "l_nuq_epistemic": nuq_epistemic,
            "l_nuq_total": nuq_total,
        }
    elif method=='decomposing_md':
        disc_md = lambda x: np.squeeze(x[0], axis=-1)
        nondisc_md = lambda x: np.squeeze(x[1], axis=-1)
        sum_md = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {"disc_md": disc_md, 
                       'nondisc_md': nondisc_md,
                       'disc+nondisc_md': sum_md}
    elif method=='mahalanobis':
        maha_dist = lambda x: np.squeeze(x[0], axis=-1)
        rel_maha_dist = lambda x: np.squeeze(x[1], axis=-1)
        marg_maha_dist = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {"mahalanobis_distance": maha_dist,
                       "relative_mahalanobis_distance": rel_maha_dist,
                       "marginal_mahalanobis_distance": marg_maha_dist}
    elif method=='l_mahalanobis':
        maha_dist = lambda x: np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"l_mahalanobis_distance": maha_dist}
    elif method=='ddu' or method=='ddu_maha':
        ddu = lambda x: -np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"ddu": ddu}
    elif method=='sngp':
        sngp = lambda x: np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"sngp": sngp}
    elif method=='mc_mahalanobis':
        sm_maha_dist = lambda x: np.squeeze(x[:, 1:], axis=-1).max(1)
        agg_methods = {"sampled_mahalanobis_distance": sm_maha_dist}
    return agg_methods

metric_types=["roc-auc", 'pr-auc', 'f1-score', 'precision', 'recall']

methods = ['l_nuq', 'nuq', 'mahalanobis', 'l_mahalanobis', 'decomposing_md', 'ddu']
regs = ['raw']
spectralnorm = ['sn', 'no_sn']
dataset_names = ['IMDB', 'TREC', 'WMT16', 'Amazon', 'MNLI', 'RTE', 'SST-2']
dataset_fnames = ['imdb', 'trec', 'wmt16', 'amazon', 'mnli', 'rte', 'sst2']
names = []
tables = []
baselines = []
for method in methods:
    for reg in regs:
        for sn in spectralnorm:
            run_dirs = []
            name_sn = ''
            names = [f'{method}|{reg}|{sn}']
            for name in dataset_fnames:
                model_series_dir = f'../../workdir/run_tasks_for_model_series/electra_{reg}_{sn}/20newsgroups/{name}/{method}'
                print(model_series_dir)
                run_dirs.append([model_series_dir])
            agg_func = choose_agg_func(method)
            try:
                res_df = collect_datasets(run_dirs, names, dataset_names, metric_types=metric_types, baselines={}, methods=agg_func)
                baselines.append(res_df.iloc[-1:])
                tables.append(res_df.iloc[:-1])
            except:
                print('pass')
                pass

../../workdir/run_tasks_for_model_series/electra_raw_sn/20newsgroups/imdb/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn/20newsgroups/trec/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn/20newsgroups/wmt16/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn/20newsgroups/amazon/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn/20newsgroups/mnli/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn/20newsgroups/rte/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn/20newsgroups/sst2/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_no_sn/20newsgroups/imdb/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_no_sn/20newsgroups/trec/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_no_sn/20newsgroups/wmt16/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_no_sn/20newsgroups/amazon/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_no_sn/20newsgroups/mnli/l_nuq
../../workdir/run_ta

In [32]:
table_all = pd.concat([pd.concat(tables), pd.concat(baselines[-2:])]).reset_index()

In [33]:
def preproc_regs(x):
    reg = x.split('|')[1]
    if reg == 'reg':
        return 'CER'
    elif reg == 'raw':
        return '-'
    else:
        return reg
    
def preproc_method(x):
    method = x.split('|')[0]
    sn = x.split('|')[-1]
    if method == 'mahalanobis' and not 'no_sn' in sn:
        return 'MD SN (ours)'
    elif method == 'mahalanobis':
        return 'MD'
    elif method == 'l_mahalanobis' and not 'no_sn' in sn:
        return 'L-MD SN (ours)'
    elif method == 'l_mahalanobis':
        return 'L-MD'
    elif method == 'mc_mahalanobis' and not 'no_sn' in sn:
        return 'SMD SN (ours)'
    elif method == 'mc_mahalanobis':
        return 'SMD'
    elif method == 'nuq' and not 'no_sn' in sn:
        return 'NUQ SN'
    elif method == 'nuq':
        return 'NUQ'
    elif method == 'l_nuq' and not 'no_sn' in sn:
        return 'L-NUQ SN'
    elif method == 'l_nuq':
        return 'L-NUQ'
    elif method == 'sngp':
        return 'SNGP'
    
    elif method == 'decomposing_md' and not 'no_sn' in sn:
        return 'Decomposing SN'
    elif method == 'decomposing_md':
        return 'Decomposing'
    
    elif method == 'nuq_best1' and not 'no_sn' in sn:
        return 'Best1 NUQ SN'
    elif method == 'nuq_best1':
        return 'Best1 NUQ'
    
    elif method == 'ddu' and not 'no_sn' in sn:
        return 'DDU SN'
    elif method == 'ddu':
        return 'DDU'
    
    elif method == 'ddu' and not 'no_sn' in sn:
        return 'DDU SN'
    elif method == 'ddu':
        return 'DDU'
    
    elif method == 'ddu_maha' and not 'no_sn' in sn:
        return 'DDU Maha SN'
    elif method == 'ddu_maha':
        return 'DDU Maha'
    
    elif 'ddpp_dpp' in method:
        return 'DDPP (+DPP) (ours)'
    elif 'ddpp_ood' in method:
        return 'DDPP (+OOD) (ours)'
    elif 'mc_all' in method:
        return 'MC dropout'
    elif 'Deep' in method:
        return 'Deep Ensemble'
    elif 'baseline|raw_no_sn' in x:
        return 'SR (baseline)'
    elif 'baseline' in x and not 'no_sn' in x:
        return 'SR SN'
    return 'SR'

def preproc_ue(x):
    if x == 'bald':
        return 'BALD'
    elif 'sampled_mahalanobis_distance' in x:
        return 'SMD'
    elif 'relative_mahalanobis_distance' in x:
        return 'rel_MD'
    elif 'marginal_mahalanobis_distance' in x:
        return 'marg_MD'
    elif 'mahalanobis_distance' in x:
        return 'MD'
    elif 'sampled_max_prob' in x:
        return 'SMP'
    elif 'variance' in x:
        return 'PV'
    elif 'aleatoric' in x:
        return 'aleatoric'
    elif 'epistemic' in x:
        return 'epistemic'
    elif 'total' in x:
        return 'total'
    elif x == 'disc_md':
        return 'Disc MD'
    elif x == 'nondisc_md':
        return 'Nondisc MD'
    elif x == 'disc+nondisc_md':
        return 'Disc+Nondisc MD'
    elif x == 'sngp':
        return 'std'
    return 'MP'

table_all['Reg. Type'] = table_all.Method.apply(lambda x: preproc_regs(x))
table_all['Method'] = table_all.Method.apply(lambda x: preproc_method(x))
table_all['UE Score'] = table_all['UE Score'].apply(lambda x: preproc_ue(x))
table_all = table_all[list(table_all.columns[:1]) + list(table_all.columns[-1:]) + list(table_all.columns[1:-1])].reset_index(drop=True)

In [34]:
roc_auc_mean = table_all[table_all.columns[3::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
roc_auc_std = table_all[table_all.columns[3::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

f1_mean = table_all[table_all.columns[5::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
f1_std = table_all[table_all.columns[5::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

pr_auc_mean = table_all[table_all.columns[4::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
pr_auc_std = table_all[table_all.columns[4::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

pr_mean = table_all[table_all.columns[6::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
pr_std = table_all[table_all.columns[6::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

rec_auc_mean = table_all[table_all.columns[7::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
rec_auc_std = table_all[table_all.columns[7::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

table_all[('Mean',  'pr-auc')] = pr_auc_mean.round(2).astype(str)+'±'+pr_auc_std.round(2).astype(str)
table_all[('Mean',  'f1-score')] = f1_mean.round(2).astype(str)+'±'+f1_std.round(2).astype(str)
table_all[('Mean',  'precision')] = pr_mean.round(2).astype(str)+'±'+pr_std.round(2).astype(str)
table_all[('Mean',  'recall')] = rec_auc_mean.round(2).astype(str)+'±'+rec_auc_std.round(2).astype(str)
table_all[('Mean',  'roc-auc')] = roc_auc_mean.round(2).astype(str)+'±'+roc_auc_std.round(2).astype(str)

In [35]:
def bold_max(table):
    attr = 'font-weight: bold'
    data = table[table.columns[3:]].apply(lambda x: x.str.split('±').str[0].astype(float))
    is_max = data == data.max()
    info_col = table[table.columns[:3]]
    for col in info_col.columns:
        info_col[col] = ''
    vals = pd.DataFrame(np.where(is_max, attr, ''),
                        index=data.index, columns=data.columns)
    return pd.concat([info_col, vals], axis=1)

def highlight_nmax(s):
    try:
        s_vals = s.str.split('±').str[0].astype(float)
        is_large = s_vals.nlargest(3).values
        style_str = ['background-color: yellow' if v in is_large else '' for v in s_vals]
        if len(np.unique(style_str))==1:
            return ['']*len(style_str)
        return style_str
    except:
        return ['']*len(s)

In [36]:
table_all[list(table_all.columns[:3]) + list(table_all.columns[-5:])].style.apply(highlight_nmax).apply(bold_max, axis=None)

/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [37]:
table_all.style.apply(highlight_nmax).apply(bold_max, axis=None)

/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


## SN ALL

In [21]:
import os 

def choose_agg_func(method):
    agg_methods = {
        "bald": bald,
        "sampled_max_prob": sampled_max_prob,
        "variance": probability_variance,
    }
    if method=='nuq' or method=='nuq_best' or method=='nuq_best1':
        nuq_aleatoric = lambda x: np.squeeze(x[0], axis=-1)
        nuq_epistemic = lambda x: np.squeeze(x[1], axis=-1)
        nuq_total = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {
            "nuq_aleatoric": nuq_aleatoric,
            "nuq_epistemic": nuq_epistemic,
            "nuq_total": nuq_total,
        }
    elif method=='l_nuq':
        nuq_aleatoric = lambda x: np.squeeze(x[0], axis=-1)
        nuq_epistemic = lambda x: np.squeeze(x[1], axis=-1)
        nuq_total = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {
            "l_nuq_aleatoric": nuq_aleatoric,
            "l_nuq_epistemic": nuq_epistemic,
            "l_nuq_total": nuq_total,
        }
    elif method=='decomposing_md':
        disc_md = lambda x: np.squeeze(x[0], axis=-1)
        nondisc_md = lambda x: np.squeeze(x[1], axis=-1)
        sum_md = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {"disc_md": disc_md, 
                       'nondisc_md': nondisc_md,
                       'disc+nondisc_md': sum_md}
    elif method=='mahalanobis':
        maha_dist = lambda x: np.squeeze(x[0], axis=-1)
        rel_maha_dist = lambda x: np.squeeze(x[1], axis=-1)
        marg_maha_dist = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {"mahalanobis_distance": maha_dist,
                       "relative_mahalanobis_distance": rel_maha_dist,
                       "marginal_mahalanobis_distance": marg_maha_dist}
    elif method=='l_mahalanobis':
        maha_dist = lambda x: np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"l_mahalanobis_distance": maha_dist}
    elif method=='ddu' or method=='ddu_maha':
        ddu = lambda x: -np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"ddu": ddu}
    elif method=='sngp':
        sngp = lambda x: np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"sngp": sngp}
    elif method=='mc_mahalanobis':
        sm_maha_dist = lambda x: np.squeeze(x[:, 1:], axis=-1).max(1)
        agg_methods = {"sampled_mahalanobis_distance": sm_maha_dist}
    return agg_methods
metric_types=["roc-auc", 'pr-auc', 'f1-score', 'precision', 'recall']

methods = ['l_nuq', 'nuq', 'mahalanobis', 'l_mahalanobis', 'decomposing_md', 'ddu']
regs = ['raw']
spectralnorm = ['sn_all']
dataset_names = ['IMDB', 'TREC', 'WMT16', 'Amazon', 'MNLI', '20newsgroups', 'RTE']
dataset_fnames = ['imdb', 'trec', 'wmt16', 'amazon', 'mnli', 'newsgroup', 'rte']
names = []
tables = []
baselines = []
for method in methods:
    for reg in regs:
        for sn in spectralnorm:
            run_dirs = []
            name_sn = ''
            names = [f'{method}|{reg}|{sn}']
            for name in dataset_fnames:
                model_series_dir = f'../../workdir/run_tasks_for_model_series/electra_{reg}_{sn}/sst2/{name}/{method}'
                print(model_series_dir)
                run_dirs.append([model_series_dir])
            agg_func = choose_agg_func(method)
            try:
                res_df = collect_datasets(run_dirs, names, dataset_names, metric_types=metric_types, baselines={}, methods=agg_func)
                baselines.append(res_df.iloc[-1:])
                tables.append(res_df.iloc[:-1])
            except:
                print('pass')
                pass

../../workdir/run_tasks_for_model_series/electra_raw_sn_all/sst2/imdb/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/sst2/trec/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/sst2/wmt16/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/sst2/amazon/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/sst2/mnli/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/sst2/newsgroup/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/sst2/rte/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/sst2/imdb/nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/sst2/trec/nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/sst2/wmt16/nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/sst2/amazon/nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/sst2/mnli/nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/sst2/newsgroup/nuq
../../wor

In [22]:
table_all = pd.concat([pd.concat(tables), pd.concat(baselines[-1:])]).reset_index()

In [23]:
def preproc_regs(x):
    reg = x.split('|')[1]
    if reg == 'reg':
        return 'CER'
    elif reg == 'raw':
        return '-'
    else:
        return reg
    
def preproc_method(x):
    method = x.split('|')[0]
    sn = x.split('|')[-1]
    if method == 'mahalanobis' and not 'no_sn' in sn:
        return 'MD SN_ALL (ours)'
    elif method == 'mahalanobis':
        return 'MD'
    elif method == 'l_mahalanobis' and not 'no_sn' in sn:
        return 'L-MD SN_ALL (ours)'
    elif method == 'l_mahalanobis':
        return 'L-MD'
    elif method == 'mc_mahalanobis' and not 'no_sn' in sn:
        return 'SMD SN_ALL (ours)'
    elif method == 'mc_mahalanobis':
        return 'SMD'
    elif method == 'nuq' and not 'no_sn' in sn:
        return 'NUQ SN_ALL'
    elif method == 'nuq':
        return 'NUQ'
    elif method == 'l_nuq' and not 'no_sn' in sn:
        return 'L-NUQ SN_ALL'
    elif method == 'l_nuq':
        return 'L-NUQ'
    elif method == 'sngp':
        return 'SNGP'
    
    elif method == 'decomposing_md' and not 'no_sn' in sn:
        return 'Decomposing SN_ALL'
    elif method == 'decomposing_md':
        return 'Decomposing'
    
    elif method == 'nuq_best1' and not 'no_sn' in sn:
        return 'Best1 NUQ SN_ALL'
    elif method == 'nuq_best1':
        return 'Best1 NUQ'
    
    elif method == 'ddu' and not 'no_sn' in sn:
        return 'DDU SN_ALL'
    elif method == 'ddu':
        return 'DDU'
    
    elif method == 'ddu_maha' and not 'no_sn' in sn:
        return 'DDU Maha SN_ALL'
    elif method == 'ddu_maha':
        return 'DDU Maha'
    
    elif 'ddpp_dpp' in method:
        return 'DDPP (+DPP) (ours)'
    elif 'ddpp_ood' in method:
        return 'DDPP (+OOD) (ours)'
    elif 'mc_all' in method:
        return 'MC dropout'
    elif 'Deep' in method:
        return 'Deep Ensemble'
    elif 'baseline|raw_no_sn' in x:
        return 'SR (baseline)'
    elif 'baseline' in x and not 'no_sn' in x:
        return 'SR SN'
    return 'SR'

def preproc_ue(x):
    if x == 'bald':
        return 'BALD'
    elif 'sampled_mahalanobis_distance' in x:
        return 'SMD'
    elif 'relative_mahalanobis_distance' in x:
        return 'rel_MD'
    elif 'marginal_mahalanobis_distance' in x:
        return 'marg_MD'
    elif 'mahalanobis_distance' in x:
        return 'MD'
    elif 'sampled_max_prob' in x:
        return 'SMP'
    elif 'variance' in x:
        return 'PV'
    elif 'aleatoric' in x:
        return 'aleatoric'
    elif 'epistemic' in x:
        return 'epistemic'
    elif 'total' in x:
        return 'total'
    elif x == 'disc_md':
        return 'Disc MD'
    elif x == 'nondisc_md':
        return 'Nondisc MD'
    elif x == 'disc+nondisc_md':
        return 'Disc+Nondisc MD'
    elif x == 'sngp':
        return 'std'
    return 'MP'

table_all['Reg. Type'] = table_all.Method.apply(lambda x: preproc_regs(x))
table_all['Method'] = table_all.Method.apply(lambda x: preproc_method(x))
table_all['UE Score'] = table_all['UE Score'].apply(lambda x: preproc_ue(x))
table_all = table_all[list(table_all.columns[:1]) + list(table_all.columns[-1:]) + list(table_all.columns[1:-1])].reset_index(drop=True)

In [24]:
roc_auc_mean = table_all[table_all.columns[3::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
roc_auc_std = table_all[table_all.columns[3::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

f1_mean = table_all[table_all.columns[5::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
f1_std = table_all[table_all.columns[5::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

pr_auc_mean = table_all[table_all.columns[4::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
pr_auc_std = table_all[table_all.columns[4::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

pr_mean = table_all[table_all.columns[6::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
pr_std = table_all[table_all.columns[6::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

rec_auc_mean = table_all[table_all.columns[7::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
rec_auc_std = table_all[table_all.columns[7::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

table_all[('Mean',  'pr-auc')] = pr_auc_mean.round(2).astype(str)+'±'+pr_auc_std.round(2).astype(str)
table_all[('Mean',  'f1-score')] = f1_mean.round(2).astype(str)+'±'+f1_std.round(2).astype(str)
table_all[('Mean',  'precision')] = pr_mean.round(2).astype(str)+'±'+pr_std.round(2).astype(str)
table_all[('Mean',  'recall')] = rec_auc_mean.round(2).astype(str)+'±'+rec_auc_std.round(2).astype(str)
table_all[('Mean',  'roc-auc')] = roc_auc_mean.round(2).astype(str)+'±'+roc_auc_std.round(2).astype(str)

In [25]:
def bold_max(table):
    attr = 'font-weight: bold'
    data = table[table.columns[3:]].apply(lambda x: x.str.split('±').str[0].astype(float))
    is_max = data == data.max()
    info_col = table[table.columns[:3]]
    for col in info_col.columns:
        info_col[col] = ''
    vals = pd.DataFrame(np.where(is_max, attr, ''),
                        index=data.index, columns=data.columns)
    return pd.concat([info_col, vals], axis=1)

def highlight_nmax(s):
    try:
        s_vals = s.str.split('±').str[0].astype(float)
        is_large = s_vals.nlargest(3).values
        style_str = ['background-color: yellow' if v in is_large else '' for v in s_vals]
        if len(np.unique(style_str))==1:
            return ['']*len(style_str)
        return style_str
    except:
        return ['']*len(s)

In [26]:
table_all[list(table_all.columns[:3]) + list(table_all.columns[-5:])].style.apply(highlight_nmax).apply(bold_max, axis=None)

/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


#### baseline - 75.56±5.49
#### best - 92.64±1.23

In [27]:
import os 

def choose_agg_func(method):
    agg_methods = {
        "bald": bald,
        "sampled_max_prob": sampled_max_prob,
        "variance": probability_variance,
    }
    if method=='nuq' or method=='nuq_best' or method=='nuq_best1':
        nuq_aleatoric = lambda x: np.squeeze(x[0], axis=-1)
        nuq_epistemic = lambda x: np.squeeze(x[1], axis=-1)
        nuq_total = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {
            "nuq_aleatoric": nuq_aleatoric,
            "nuq_epistemic": nuq_epistemic,
            "nuq_total": nuq_total,
        }
    elif method=='l_nuq':
        nuq_aleatoric = lambda x: np.squeeze(x[0], axis=-1)
        nuq_epistemic = lambda x: np.squeeze(x[1], axis=-1)
        nuq_total = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {
            "l_nuq_aleatoric": nuq_aleatoric,
            "l_nuq_epistemic": nuq_epistemic,
            "l_nuq_total": nuq_total,
        }
    elif method=='decomposing_md':
        disc_md = lambda x: np.squeeze(x[0], axis=-1)
        nondisc_md = lambda x: np.squeeze(x[1], axis=-1)
        sum_md = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {"disc_md": disc_md, 
                       'nondisc_md': nondisc_md,
                       'disc+nondisc_md': sum_md}
    elif method=='mahalanobis':
        maha_dist = lambda x: np.squeeze(x[0], axis=-1)
        rel_maha_dist = lambda x: np.squeeze(x[1], axis=-1)
        marg_maha_dist = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {"mahalanobis_distance": maha_dist,
                       "relative_mahalanobis_distance": rel_maha_dist,
                       "marginal_mahalanobis_distance": marg_maha_dist}
    elif method=='l_mahalanobis':
        maha_dist = lambda x: np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"l_mahalanobis_distance": maha_dist}
    elif method=='ddu' or method=='ddu_maha':
        ddu = lambda x: -np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"ddu": ddu}
    elif method=='sngp':
        sngp = lambda x: np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"sngp": sngp}
    elif method=='mc_mahalanobis':
        sm_maha_dist = lambda x: np.squeeze(x[:, 1:], axis=-1).max(1)
        agg_methods = {"sampled_mahalanobis_distance": sm_maha_dist}
    return agg_methods
metric_types=["roc-auc", 'pr-auc', 'f1-score', 'precision', 'recall']

methods = ['l_nuq', 'nuq', 'mahalanobis', 'l_mahalanobis', 'decomposing_md', 'ddu']
regs = ['raw']
spectralnorm = ['sn_all']
dataset_names = ['IMDB', 'TREC', 'WMT16', 'Amazon', 'MNLI', 'SST-2', 'RTE']
dataset_fnames = ['imdb', 'trec', 'wmt16', 'amazon', 'mnli', 'sst2', 'rte']
names = []
tables = []
baselines = []
for method in methods:
    for reg in regs:
        for sn in spectralnorm:
            run_dirs = []
            name_sn = ''
            names = [f'{method}|{reg}|{sn}']
            for name in dataset_fnames:
                model_series_dir = f'../../workdir/run_tasks_for_model_series/electra_{reg}_{sn}/20newsgroups/{name}/{method}'
                print(model_series_dir)
                run_dirs.append([model_series_dir])
            agg_func = choose_agg_func(method)
            try:
                res_df = collect_datasets(run_dirs, names, dataset_names, metric_types=metric_types, baselines={}, methods=agg_func)
                baselines.append(res_df.iloc[-1:])
                tables.append(res_df.iloc[:-1])
            except:
                print('pass')
                pass

../../workdir/run_tasks_for_model_series/electra_raw_sn_all/20newsgroups/imdb/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/20newsgroups/trec/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/20newsgroups/wmt16/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/20newsgroups/amazon/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/20newsgroups/mnli/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/20newsgroups/sst2/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/20newsgroups/rte/l_nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/20newsgroups/imdb/nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/20newsgroups/trec/nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/20newsgroups/wmt16/nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/20newsgroups/amazon/nuq
../../workdir/run_tasks_for_model_series/electra_raw_sn_all/20newsgroups/mnli/n

In [28]:
table_all = pd.concat([pd.concat(tables), pd.concat(baselines[-1:])]).reset_index()

In [29]:
def preproc_regs(x):
    reg = x.split('|')[1]
    if reg == 'reg':
        return 'CER'
    elif reg == 'raw':
        return '-'
    else:
        return reg
    
def preproc_method(x):
    method = x.split('|')[0]
    sn = x.split('|')[-1]
    if method == 'mahalanobis' and not 'no_sn' in sn:
        return 'MD SN_ALL (ours)'
    elif method == 'mahalanobis':
        return 'MD'
    elif method == 'l_mahalanobis' and not 'no_sn' in sn:
        return 'L-MD SN_ALL (ours)'
    elif method == 'l_mahalanobis':
        return 'L-MD'
    elif method == 'mc_mahalanobis' and not 'no_sn' in sn:
        return 'SMD SN_ALL (ours)'
    elif method == 'mc_mahalanobis':
        return 'SMD'
    elif method == 'nuq' and not 'no_sn' in sn:
        return 'NUQ SN_ALL'
    elif method == 'nuq':
        return 'NUQ'
    elif method == 'l_nuq' and not 'no_sn' in sn:
        return 'L-NUQ SN_ALL'
    elif method == 'l_nuq':
        return 'L-NUQ'
    elif method == 'sngp':
        return 'SNGP'
    
    elif method == 'decomposing_md' and not 'no_sn' in sn:
        return 'Decomposing SN_ALL'
    elif method == 'decomposing_md':
        return 'Decomposing'
    
    elif method == 'nuq_best1' and not 'no_sn' in sn:
        return 'Best1 NUQ SN_ALL'
    elif method == 'nuq_best1':
        return 'Best1 NUQ'
    
    elif method == 'ddu' and not 'no_sn' in sn:
        return 'DDU SN_ALL'
    elif method == 'ddu':
        return 'DDU'
    
    elif method == 'ddu_maha' and not 'no_sn' in sn:
        return 'DDU Maha SN_ALL'
    elif method == 'ddu_maha':
        return 'DDU Maha'
    
    elif 'ddpp_dpp' in method:
        return 'DDPP (+DPP) (ours)'
    elif 'ddpp_ood' in method:
        return 'DDPP (+OOD) (ours)'
    elif 'mc_all' in method:
        return 'MC dropout'
    elif 'Deep' in method:
        return 'Deep Ensemble'
    elif 'baseline|raw_no_sn' in x:
        return 'SR (baseline)'
    elif 'baseline' in x and not 'no_sn' in x:
        return 'SR SN'
    return 'SR'

def preproc_ue(x):
    if x == 'bald':
        return 'BALD'
    elif 'sampled_mahalanobis_distance' in x:
        return 'SMD'
    elif 'relative_mahalanobis_distance' in x:
        return 'rel_MD'
    elif 'marginal_mahalanobis_distance' in x:
        return 'marg_MD'
    elif 'mahalanobis_distance' in x:
        return 'MD'
    elif 'sampled_max_prob' in x:
        return 'SMP'
    elif 'variance' in x:
        return 'PV'
    elif 'aleatoric' in x:
        return 'aleatoric'
    elif 'epistemic' in x:
        return 'epistemic'
    elif 'total' in x:
        return 'total'
    elif x == 'disc_md':
        return 'Disc MD'
    elif x == 'nondisc_md':
        return 'Nondisc MD'
    elif x == 'disc+nondisc_md':
        return 'Disc+Nondisc MD'
    elif x == 'sngp':
        return 'std'
    return 'MP'

table_all['Reg. Type'] = table_all.Method.apply(lambda x: preproc_regs(x))
table_all['Method'] = table_all.Method.apply(lambda x: preproc_method(x))
table_all['UE Score'] = table_all['UE Score'].apply(lambda x: preproc_ue(x))
table_all = table_all[list(table_all.columns[:1]) + list(table_all.columns[-1:]) + list(table_all.columns[1:-1])].reset_index(drop=True)

In [30]:
roc_auc_mean = table_all[table_all.columns[3::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
roc_auc_std = table_all[table_all.columns[3::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

f1_mean = table_all[table_all.columns[5::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
f1_std = table_all[table_all.columns[5::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

pr_auc_mean = table_all[table_all.columns[4::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
pr_auc_std = table_all[table_all.columns[4::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

pr_mean = table_all[table_all.columns[6::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
pr_std = table_all[table_all.columns[6::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

rec_auc_mean = table_all[table_all.columns[7::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
rec_auc_std = table_all[table_all.columns[7::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

table_all[('Mean',  'pr-auc')] = pr_auc_mean.round(2).astype(str)+'±'+pr_auc_std.round(2).astype(str)
table_all[('Mean',  'f1-score')] = f1_mean.round(2).astype(str)+'±'+f1_std.round(2).astype(str)
table_all[('Mean',  'precision')] = pr_mean.round(2).astype(str)+'±'+pr_std.round(2).astype(str)
table_all[('Mean',  'recall')] = rec_auc_mean.round(2).astype(str)+'±'+rec_auc_std.round(2).astype(str)
table_all[('Mean',  'roc-auc')] = roc_auc_mean.round(2).astype(str)+'±'+roc_auc_std.round(2).astype(str)

In [31]:
def bold_max(table):
    attr = 'font-weight: bold'
    data = table[table.columns[3:]].apply(lambda x: x.str.split('±').str[0].astype(float))
    is_max = data == data.max()
    info_col = table[table.columns[:3]]
    for col in info_col.columns:
        info_col[col] = ''
    vals = pd.DataFrame(np.where(is_max, attr, ''),
                        index=data.index, columns=data.columns)
    return pd.concat([info_col, vals], axis=1)

def highlight_nmax(s):
    try:
        s_vals = s.str.split('±').str[0].astype(float)
        is_large = s_vals.nlargest(3).values
        style_str = ['background-color: yellow' if v in is_large else '' for v in s_vals]
        if len(np.unique(style_str))==1:
            return ['']*len(style_str)
        return style_str
    except:
        return ['']*len(s)

In [32]:
table_all[list(table_all.columns[:3]) + list(table_all.columns[-5:])].style.apply(highlight_nmax).apply(bold_max, axis=None)

/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


#### baseline - 81.06±3.09
#### best - 95.91±0.87

In [51]:
import os 

def choose_agg_func(method):
    agg_methods = {
        "bald": bald,
        "sampled_max_prob": sampled_max_prob,
        "variance": probability_variance,
    }
    if method=='nuq' or method=='nuq_best' or method=='nuq_best1':
        nuq_aleatoric = lambda x: np.squeeze(x[0], axis=-1)
        nuq_epistemic = lambda x: np.squeeze(x[1], axis=-1)
        nuq_total = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {
            "nuq_aleatoric": nuq_aleatoric,
            "nuq_epistemic": nuq_epistemic,
            "nuq_total": nuq_total,
        }
    elif method=='l_nuq':
        nuq_aleatoric = lambda x: np.squeeze(x[0], axis=-1)
        nuq_epistemic = lambda x: np.squeeze(x[1], axis=-1)
        nuq_total = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {
            "l_nuq_aleatoric": nuq_aleatoric,
            "l_nuq_epistemic": nuq_epistemic,
            "l_nuq_total": nuq_total,
        }
    elif method=='decomposing_md':
        disc_md = lambda x: np.squeeze(x[0], axis=-1)
        nondisc_md = lambda x: np.squeeze(x[1], axis=-1)
        sum_md = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {"disc_md": disc_md, 
                       'nondisc_md': nondisc_md,
                       'disc+nondisc_md': sum_md}
    elif method=='mahalanobis':
        maha_dist = lambda x: np.squeeze(x[:, 0], axis=-1)
        #rel_maha_dist = lambda x: np.squeeze(x[1], axis=-1)
        #marg_maha_dist = lambda x: np.squeeze(x[2], axis=-1)
        agg_methods = {"mahalanobis_distance": maha_dist,
                       #"relative_mahalanobis_distance": rel_maha_dist,
                       #"marginal_mahalanobis_distance": marg_maha_dist
                      }
    elif method=='l_mahalanobis':
        maha_dist = lambda x: np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"l_mahalanobis_distance": maha_dist}
    elif method=='ddu' or method=='ddu_maha':
        ddu = lambda x: -np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"ddu": ddu}
    elif method=='sngp':
        sngp = lambda x: np.squeeze(x[:, 0], axis=-1)
        agg_methods = {"sngp": sngp}
    elif method=='mc_mahalanobis':
        sm_maha_dist = lambda x: np.squeeze(x[:, 1:], axis=-1).max(1)
        agg_methods = {"sampled_mahalanobis_distance": sm_maha_dist}
    return agg_methods
metric_types=["roc-auc", 'pr-auc', 'f1-score', 'precision', 'recall']

methods = ['mahalanobis', 'mc', 'nuq']
regs = ['hs_rau']
spectralnorm = ['no_sn']
dataset_names = ['IMDB', 'TREC', 'WMT16', 'Amazon', 'MNLI', '20NewsGroups', 'RTE']
dataset_fnames = ['imdb', 'trec', 'wmt16', 'amazon', 'mnli', '20newsgroups', 'rte']
names = []
tables = []
baselines = []
for method in methods:
    for reg in regs:
        for sn in spectralnorm:
            run_dirs = []
            name_sn = ''
            names = [f'{method}|{reg}|{sn}']
            for name in dataset_fnames:
                model_series_dir = f'../../../uncertainty-estimation_cp/workdir/run_tasks_for_model_series/electra_{reg}_{sn}/sst2/{name}/{method}'
                print(model_series_dir)
                run_dirs.append([model_series_dir])
            agg_func = choose_agg_func(method)
            try:
                res_df = collect_datasets(run_dirs, names, dataset_names, metric_types=metric_types, baselines={}, methods=agg_func)
                baselines.append(res_df.iloc[-1:])
                tables.append(res_df.iloc[:-1])
            except:
                print('pass')
                pass

../../../uncertainty-estimation_cp/workdir/run_tasks_for_model_series/electra_hs_rau_no_sn/sst2/imdb/mahalanobis
../../../uncertainty-estimation_cp/workdir/run_tasks_for_model_series/electra_hs_rau_no_sn/sst2/trec/mahalanobis
../../../uncertainty-estimation_cp/workdir/run_tasks_for_model_series/electra_hs_rau_no_sn/sst2/wmt16/mahalanobis
../../../uncertainty-estimation_cp/workdir/run_tasks_for_model_series/electra_hs_rau_no_sn/sst2/amazon/mahalanobis
../../../uncertainty-estimation_cp/workdir/run_tasks_for_model_series/electra_hs_rau_no_sn/sst2/mnli/mahalanobis
../../../uncertainty-estimation_cp/workdir/run_tasks_for_model_series/electra_hs_rau_no_sn/sst2/20newsgroups/mahalanobis
../../../uncertainty-estimation_cp/workdir/run_tasks_for_model_series/electra_hs_rau_no_sn/sst2/rte/mahalanobis
../../../uncertainty-estimation_cp/workdir/run_tasks_for_model_series/electra_hs_rau_no_sn/sst2/imdb/mc
../../../uncertainty-estimation_cp/workdir/run_tasks_for_model_series/electra_hs_rau_no_sn/sst2

In [52]:
table_all = pd.concat([pd.concat([t[:-1] for t in tables]), pd.concat(baselines[-1:])]).reset_index()

In [53]:
def preproc_regs(x):
    reg = x.split('|')[1]
    if reg == 'reg':
        return 'CER'
    elif reg == 'raw':
        return '-'
    elif reg == 'hs_rau':
        return 'HS-RAU'
    else:
        return reg
    
def preproc_method(x):
    method = x.split('|')[0]
    sn = x.split('|')[-1]
    if method == 'mahalanobis' and not 'no_sn' in sn:
        return 'MD SN_ALL (ours)'
    elif method == 'mahalanobis':
        return 'MD'
    elif method == 'l_mahalanobis' and not 'no_sn' in sn:
        return 'L-MD SN_ALL (ours)'
    elif method == 'l_mahalanobis':
        return 'L-MD'
    elif method == 'mc_mahalanobis' and not 'no_sn' in sn:
        return 'SMD SN_ALL (ours)'
    elif method == 'mc_mahalanobis':
        return 'SMD'
    elif method == 'nuq' and not 'no_sn' in sn:
        return 'NUQ SN_ALL'
    elif method == 'nuq':
        return 'NUQ'
    elif method == 'l_nuq' and not 'no_sn' in sn:
        return 'L-NUQ SN_ALL'
    elif method == 'l_nuq':
        return 'L-NUQ'
    elif method == 'sngp':
        return 'SNGP'
    
    elif method == 'decomposing_md' and not 'no_sn' in sn:
        return 'Decomposing SN_ALL'
    elif method == 'decomposing_md':
        return 'Decomposing'
    
    elif method == 'nuq_best1' and not 'no_sn' in sn:
        return 'Best1 NUQ SN_ALL'
    elif method == 'nuq_best1':
        return 'Best1 NUQ'
    
    elif method == 'ddu' and not 'no_sn' in sn:
        return 'DDU SN_ALL'
    elif method == 'ddu':
        return 'DDU'
    
    elif method == 'ddu_maha' and not 'no_sn' in sn:
        return 'DDU Maha SN_ALL'
    elif method == 'ddu_maha':
        return 'DDU Maha'
    
    elif 'ddpp_dpp' in method:
        return 'DDPP (+DPP) (ours)'
    elif 'ddpp_ood' in method:
        return 'DDPP (+OOD) (ours)'
    elif 'mc' in method:
        return 'MC dropout'
    elif 'Deep' in method:
        return 'Deep Ensemble'
    elif 'baseline|raw_no_sn' in x:
        return 'SR (baseline)'
    elif 'baseline' in x and not 'no_sn' in x:
        return 'SR SN'
    return 'SR'

def preproc_ue(x):
    if x == 'bald':
        return 'BALD'
    elif 'sampled_mahalanobis_distance' in x:
        return 'SMD'
    elif 'relative_mahalanobis_distance' in x:
        return 'rel_MD'
    elif 'marginal_mahalanobis_distance' in x:
        return 'marg_MD'
    elif 'mahalanobis_distance' in x:
        return 'MD'
    elif 'sampled_max_prob' in x:
        return 'SMP'
    elif 'variance' in x:
        return 'PV'
    elif 'aleatoric' in x:
        return 'aleatoric'
    elif 'epistemic' in x:
        return 'epistemic'
    elif 'total' in x:
        return 'total'
    elif x == 'disc_md':
        return 'Disc MD'
    elif x == 'nondisc_md':
        return 'Nondisc MD'
    elif x == 'disc+nondisc_md':
        return 'Disc+Nondisc MD'
    elif x == 'sngp':
        return 'std'
    return 'MP'

table_all['Reg. Type'] = table_all.Method.apply(lambda x: preproc_regs(x))
table_all['Method'] = table_all.Method.apply(lambda x: preproc_method(x))
table_all['UE Score'] = table_all['UE Score'].apply(lambda x: preproc_ue(x))
table_all = table_all[list(table_all.columns[:1]) + list(table_all.columns[-1:]) + list(table_all.columns[1:-1])].reset_index(drop=True)

In [54]:
roc_auc_mean = table_all[table_all.columns[3::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
roc_auc_std = table_all[table_all.columns[3::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

f1_mean = table_all[table_all.columns[5::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
f1_std = table_all[table_all.columns[5::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

pr_auc_mean = table_all[table_all.columns[4::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
pr_auc_std = table_all[table_all.columns[4::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

pr_mean = table_all[table_all.columns[6::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
pr_std = table_all[table_all.columns[6::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

rec_auc_mean = table_all[table_all.columns[7::5]].apply(lambda x: x.str.split('±').str[0].astype(float)).mean(axis=1)
rec_auc_std = table_all[table_all.columns[7::5]].apply(lambda x: x.str.split('±').str[1].astype(float)).mean(axis=1)

table_all[('Mean',  'pr-auc')] = pr_auc_mean.round(2).astype(str)+'±'+pr_auc_std.round(2).astype(str)
table_all[('Mean',  'f1-score')] = f1_mean.round(2).astype(str)+'±'+f1_std.round(2).astype(str)
table_all[('Mean',  'precision')] = pr_mean.round(2).astype(str)+'±'+pr_std.round(2).astype(str)
table_all[('Mean',  'recall')] = rec_auc_mean.round(2).astype(str)+'±'+rec_auc_std.round(2).astype(str)
table_all[('Mean',  'roc-auc')] = roc_auc_mean.round(2).astype(str)+'±'+roc_auc_std.round(2).astype(str)

In [55]:
table_all[list(table_all.columns[:3]) + list(table_all.columns[-5:])]

Method Reg. Type   UE Score        Mean                          \
                                        pr-auc    f1-score   precision   
0          MD    HS-RAU         MD  91.03±2.31  90.74±1.55  86.73±2.42   
1  MC dropout    HS-RAU       BALD  82.24±1.83  87.67±1.17  82.45±1.59   
2  MC dropout    HS-RAU        SMP  83.78±2.06   88.15±1.6  83.03±2.09   
3  MC dropout    HS-RAU         PV  82.55±1.87  87.88±1.28  82.79±1.88   
4         NUQ    HS-RAU  aleatoric  89.61±2.63  90.05±1.94  85.89±2.59   
5         NUQ    HS-RAU  epistemic   88.8±2.16  89.86±1.76  85.95±1.68   
6         NUQ    HS-RAU      total   89.5±2.46   90.1±1.72  85.98±2.29   
7          SR    HS-RAU         MP  87.74±2.57  88.61±2.23  85.98±2.29   

                           
       recall     roc-auc  
0  95.98±1.11  87.12±2.38  
1  94.66±1.41  73.17±2.84  
2  95.06±2.04  74.63±3.37  
3  94.75±1.84  73.71±2.97  
4  95.52±1.25   84.4±3.78  
5  94.63±3.01   82.6±3.71  
6  95.33±2.27  83.89±3.71  
7  95.33±2.27  78.53±4.15

In [56]:
table_all

Method Reg. Type   UE Score        IMDB                          \
                                       roc-auc      pr-auc    f1-score   
0          MD    HS-RAU         MD  82.18±2.00  99.04±0.14  98.39±0.04   
1  MC dropout    HS-RAU       BALD  57.20±3.13  96.60±0.18  98.29±0.00   
2  MC dropout    HS-RAU        SMP  58.89±3.66  97.24±0.21  98.29±0.02   
3  MC dropout    HS-RAU         PV  57.96±3.27  96.74±0.19  98.29±0.00   
4         NUQ    HS-RAU  aleatoric  77.77±2.89  98.76±0.15  98.37±0.05   
5         NUQ    HS-RAU  epistemic  75.60±3.15  98.61±0.18  98.37±0.04   
6         NUQ    HS-RAU      total  77.07±2.94  98.71±0.15  98.37±0.04   
7          SR    HS-RAU         MP  64.87±4.47  98.00±0.28  98.31±0.03   

                                   TREC               ...         RTE  \
    precision       recall      roc-auc       pr-auc  ...     roc-auc   
0  96.97±0.10   99.86±0.04   84.71±7.71  69.45±11.56  ...  92.61±0.92   
1  96.63±0.00  100.00±0.00   82.20±5.20   62.24±9.80  ...  79.07±1.94   
2  96.67±0.06   99.98±0.04   82.29±5.67   61.94±9.37  ...  81.32±2.98   
3  96.64±0.01  100.00±0.00   82.27±5.35   62.29±9.80  ...  79.82±2.18   
4  96.88±0.13   99.91±0.05  81.94±11.57  65.09±12.64  ...  91.09±1.58   
5  96.90±0.13   99.88±0.07   84.45±5.46   66.25±7.98  ...  88.67±3.25   
6  96.89±0.13   99.89±0.07   83.45±8.62  66.02±10.91  ...  90.29±2.26   
7  96.89±0.13   99.89±0.07   83.51±6.63   65.54±8.90  ...  86.94±4.17   

                                                         Mean              \
       pr-auc    f1-score   precision      recall      pr-auc    f1-score   
0  77.40±3.23  75.10±1.96  67.34±3.99  85.20±3.68  91.03±2.31  90.74±1.55   
1  42.07±1.60  60.65±3.01  50.16±3.20  76.90±4.75  82.24±1.83  87.67±1.17   
2  47.41±2.88  62.55±4.13  51.76±3.88  79.60±8.24  83.78±2.06   88.15±1.6   
3  43.04±1.73  61.46±3.33  50.47±3.88  79.06±6.00  82.55±1.87  87.88±1.28   
4  73.90±3.99  73.68±2.07  66.99±2.32  82.01±3.90  89.61±2.63  90.05±1.94   
5  68.90±4.81  71.35±3.81  65.23±2.73  79.36±8.74   88.8±2.16  89.86±1.76   
6  72.76±4.32  72.94±2.27  66.42±4.03  81.59±6.99   89.5±2.46   90.1±1.72   
7  64.97±6.38  66.62±6.18  66.42±4.03  81.59±6.99  87.74±2.57  88.61±2.23   

                                       
    precision      recall     roc-auc  
0  86.73±2.42  95.98±1.11  87.12±2.38  
1  82.45±1.59  94.66±1.41  73.17±2.84  
2  83.03±2.09  95.06±2.04  74.63±3.37  
3  82.79±1.88  94.75±1.84  73.71±2.97  
4  85.89±2.59  95.52±1.25   84.4±3.78  
5  85.95±1.68  94.63±3.01   82.6±3.71  
6  85.98±2.29  95.33±2.27  83.89±3.71  
7  85.98±2.29  95.33±2.27  78.53±4.15  

[8 rows x 43 columns]